In [8]:
from radiomics import featureextractor
from damply import dirs
from pathlib import Path
import SimpleITK as sitk
import pandas as pd

In [ ]:
settings = dirs.CONFIG / "pyradiomics" / "pyradiomics_original_single_feature.yaml" 
image = sitk.ReadImage(f"{dirs.PROCDATA}/TCIA_NSCLC-Radiomics/images/mit_NSCLC-Radiomics/LUNG1-001_0000/CT_63382046/CT.nii.gz")
mask = sitk.ReadImage(f"{dirs.PROCDATA}/TCIA_NSCLC-Radiomics/images/mit_NSCLC-Radiomics/LUNG1-001_0000/RTSTRUCT_35578236/GTV.nii.gz")

In [11]:
dataset_index = pd.read_csv(f"{dirs.PROCDATA}/TCIA_NSCLC-Radiomics/features/pyradiomics/pyradiomics_NSCLC-Radiomics_index.csv")

metadata = dataset_index.iloc[0]

In [5]:
# Convert settings Path to string for pyradiomics to read it
if isinstance(settings, Path):
    settings = str(settings)

try:
    # Set up PyRadiomics feature extractor with provided settings file (expects a string, not a pathlib Path)
    extractor = featureextractor.RadiomicsFeatureExtractor(settings)

    sample_feature_vector = extractor.execute(image, mask)

except Exception as e:
    print(f"Feature extraction failed for this sample: {e}")

In [15]:
from collections import OrderedDict
od_metadata = metadata.to_dict(into=OrderedDict)

In [6]:
sample_feature_vector

OrderedDict([('diagnostics_Versions_PyRadiomics', 'v3.1.4'),
             ('diagnostics_Versions_Numpy', '2.2.6'),
             ('diagnostics_Versions_SimpleITK', '2.5.0'),
             ('diagnostics_Versions_PyWavelet', '1.8.0'),
             ('diagnostics_Versions_Python', '3.12.10'),
             ('diagnostics_Configuration_Settings',
              {'minimumROIDimensions': 2,
               'minimumROISize': None,
               'normalize': False,
               'normalizeScale': 1,
               'removeOutliers': None,
               'resampledPixelSpacing': [1.0, 1.0, 1.0],
               'interpolator': 'sitkBSpline',
               'preCrop': False,
               'padDistance': 5,
               'distances': [1],
               'force2D': False,
               'force2Ddimension': 0,
               'resegmentRange': None,
               'label': 1,
               'additionalInfo': True,
               'binWidth': 25}),
             ('diagnostics_Configuration_EnabledImageTypes

In [16]:
od_metadata.update(sample_feature_vector)

In [17]:
od_metadata

OrderedDict([('SampleID', 'LUNG1-001_0000'),
             ('MaskID', 'GTV'),
             ('Permutation', 'original'),
             ('Region', 'full'),
             ('Image',
              'mit_NSCLC-Radiomics/LUNG1-001_0000/CT_63382046/CT.nii.gz'),
             ('Mask',
              'mit_NSCLC-Radiomics/LUNG1-001_0000/RTSTRUCT_35578236/GTV.nii.gz'),
             ('diagnostics_Versions_PyRadiomics', 'v3.1.4'),
             ('diagnostics_Versions_Numpy', '2.2.6'),
             ('diagnostics_Versions_SimpleITK', '2.5.0'),
             ('diagnostics_Versions_PyWavelet', '1.8.0'),
             ('diagnostics_Versions_Python', '3.12.10'),
             ('diagnostics_Configuration_Settings',
              {'minimumROIDimensions': 2,
               'minimumROISize': None,
               'normalize': False,
               'normalizeScale': 1,
               'removeOutliers': None,
               'resampledPixelSpacing': [1.0, 1.0, 1.0],
               'interpolator': 'sitkBSpline',
            